**Zadanie 1**

Wykorzystując plik z treścią Pana Tadeusza zaprezentowany w przykładach policz ile jest linii, w których zawiera się słowo Tadeusz. Wykorzystaj akcje i transformacje RDD.

In [1]:
import os


os.environ['SPARK_NAME'] = "/opt/spark"
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/venv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/venv/bin/python3'

In [2]:
from pyspark.sql import SparkSession


spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","4g")\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/16 15:04:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[2] appName=Create-DataFrame>

In [3]:
sc = spark.sparkContext
pan_tadeusz_file = sc.textFile("pan-tadeusz.txt")

In [4]:
lines_with_tadeusz = pan_tadeusz_file.filter(lambda line: 'Tadeusz' in line)
count_lines_with_tadeusz = lines_with_tadeusz.count()
print(f"Liczba linii zawierających słowo 'Tadeusz': {count_lines_with_tadeusz}")

Liczba linii zawierających słowo 'Tadeusz': 182


**Zadanie 2**  
Wykorzystując metodę `top()` dla obiektu RDD wyświetl 3 najdłuższe linie wczytane z pliku z książką Pan Tadeusz.

In [5]:
longest_lines = pan_tadeusz_file.top(3, key=lambda line: len(line))

print("3 najdłuższe linie w pliku 'Pan Tadeusz':")
for idx, line in enumerate(longest_lines, 1):
    print(f'{idx}. {line} (Długość: {len(line)})')

3 najdłuższe linie w pliku 'Pan Tadeusz':
1. O niebezpieczeństwach wynikających z nieporządnego obozowania — Odsiecz niespodziana — Smutne położenie szlachty — Odwiedziny kwestarskie są wróżbą ratunku — Major Płut zbytnią zalotnością ściąga na siebie burzę — Wystrzał z krócicy, hasło boju — Czyny Kropiciela, czyny i niebezpieczeństwa Maćka — Konewka zasadzką ocala Soplicowo — Posiłki jezdne, atak na piechotę — Czyny Tadeusza — Pojedynek dowódców przerwany zdradą — Wojski stanowczym manewrem przechyla szalę boju — Czyny krwawe Gerwazego — Podkomorzy zwycięzca wspaniałomyślny. (Długość: 536)
2. Plany myśliwskie Telimeny — Ogrodniczka wybiera się na wielki świat i słucha nauk opiekunki — Strzelcy wracają — Wielkie zadziwienie Tadeusza — Spotkanie się powtórne w Świątyni dumania i zgoda ułatwiona za pośrednictwem mrówek — U stołu wytacza się rzecz o łowach — Powieść Wojskiego o Rejtanie i księciu Denassów, przerwana — Zagajenie układów między stronami, także przerwane — Zjawisko z kluczem 

**Zadanie 3**  
Wykorzystując listę stopwords z adresu https://github.com/bieli/stopwords/blob/master/polish.stopwords.txt wykorzystaj akcje i transformacje RDD i wygeneruj listę unikalnych słów z pliku z treścią Pana Tadeusza (sprawdź transformację countByValue()) pomijając powyższe słowa stop oraz wszelkie znaki przestankowe. Wynik zapisz do słownika, a następnie do pliku json o nazwie pan_tadeusz_bag_of_words.json.
Które słowo występuje w tym tekście najczęściej? Wyświetl je z wyników wygenerowanych powyżej.

In [6]:
import json
import string

with open("polish.stopwords.txt", "r", encoding="utf-8") as f:
    stopwords = set(f.read().splitlines())

custom_punctuation = string.punctuation + "—"

words = (
    pan_tadeusz_file
    .flatMap(lambda line: line.lower().translate(str.maketrans('', '', custom_punctuation)).split())
    .filter(lambda word: word not in stopwords)
)

word_counts = words.countByValue()

with open("pan_tadeusz_bag_of_words.json", "w", encoding="utf-8") as f:
    json.dump(word_counts, f, ensure_ascii=False, indent=4)

most_frequent_word = max(word_counts, key=word_counts.get)
most_frequent_count = word_counts[most_frequent_word]

print(f"Najczęściej występujące słowo: '{most_frequent_word}' z liczbą wystąpień: {most_frequent_count}")

Najczęściej występujące słowo: 'rzekł' z liczbą wystąpień: 155
